# Data-Aid
### Transactional Analysis Project at a Glance

In [3]:
#import nltk; nltk.download('stopwords')
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
from IPython.display import HTML, display
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Initial Analysis of Data 

The initial steps to any KDD process is the data selection and exploration stage. The data in this project outlines transactions conducted in on an eCommerce site from the UCI online retail repository. (See references in report for data source)

There were informative and insightful visualisations created within this initial analysis which can be seen below:

In [19]:
display(HTML("<h2>Distribution of Transactions over Time and number of Items Bought Per Customer</h2><table><tr><td><img src='DistOfPurchasesOverTimeHistogram.png'></td></tr><tr><td><img src='CustomersNItems.png'></td></tr><tr><td><img src='Top10ItemsSold.png'></td></tr></table>"))

""
""
""


From these visualisations it can be seen that the times when transactions occur most frequently are in the early afternoon, following a normal distrubution based on the bell shaped curve of the graph

In regards to the number of items per customer, it can be seen that for most customers, they buy a relatively small set of items ( most below 20 items). There are a sigbnificant number of outliers with some customers ordering up to 100 items. This could perhaps identify a wholeseller market of items. It can also be seen what were the most common items bought by customers,the white t-shirt holder and regency cheesecake stand being the most commonly bought items. 
This is a good initial example of how useful insights can be drawn from this type of data. 

## Association Rule Mining

This is a brief overview of the results from the assocaiation rule mining on the UCI receipt data. This was conducted in R using the apriori library; an R interface of the apriori algorithm.

In [17]:
display(HTML("<h2>Association Rules Visualisations</h2><table><tr><td><img src='ScatterPlotAssocRules.png'></td></tr><tr><td><img src='GraphedRulesAssoc.png'></td></tr></table>"))

""
""


Here are visual results of the association Rule mining. The first visualisation shows the top 10 rules sorted by confidence, grpahed against that rules support. For more on these metrics see Implementation section Association rules. The second visualisation shows how the differet objects are linked to each other. For example, an expected one is sugar jars, black tea and coffee being bought together. An unexpected one would be the grouping of wobbly chicken and wobbly rabbit toys being pruchased along with metal and decorations. This information can be used by a business to explore new opportunities to increase sales and revenues on certain items. 

## Computer Vision on Receipt Data

One requirement of the project was to implement the a system where an image of a invoice or receipt is taken as an input. This image is examined using Computer Vision algorithms implemented through the Google Vision API. The textual data is extracted from the image to obtain key data about the data including:

- The date of the transaction
- The store/website where the transaction occured
- The description, quantity and cost of each line item
- the total amount of the transaction

Here is an example of a couple of images taken from the Expressway receipt database:

In [27]:
display(HTML("<h4>Sample of Receipt Images</h4><table><tr><td><img src='ReceiptImages/1001-receipt.jpg'></td><td><img src='ReceiptImages/1002-receipt.jpg'></td></tr></table>"))

,


These images illustrate a number of difficulties associated with creating a text extraction system which is both robust and scalable. While there are similarities between the structure of these receipts and similar data is shown, they are formatted in two distinct ways.
The receipts themselves are placed in different points within each image. The second image has quite alot of noise in the background with the food behind the reciept. They also have different fonts for their logos; the 'Grand-Lux cafe' is drastically different to the 'Taco Bell' name in terms of where they are on the receipt as well as how they are presented visually. 
The Google Vision API can solve some of these issues by identifying the receipt itself as an object on the image, and subsequently identify the text within that object.
Here is an example of how the Google Vision API can view the data:

In [35]:
display(HTML("<h4>Sample of Receipt Images</h4><table><tr><th>Image</th><th>Extracted Text</th></tr><tr><td><img src='Images/schuh.jpeg'></td><td><p> schuh</p><p>47-48 OConnell Street</p><p>Dublin, D01 PK22</p><p>TEL No: 01 2476147</p><p>VAT No: 8272987T</p><p>Cashier : 011: Sophie</p><p>Assistant : 011: Sophie</p><p>** Sale **</p><p>ADI GAZELLE</p><p>:3415691024506</p><p>Barcode</p><p>Size</p><p>: UK 11</p><p>Qty: 1 @ Price : €88.00</p><p>Less Discount Amount: €8.80</p><span>STUDENT CARD 10%</p><p>€79.20</p><p>** Sale **</p><p>CONS CTAS RENEW HI</p><p>: 3415955074506</p><p>: UK 11</p><p>Qty: 1 @ Price : €69.00</p><p>Less Discount Amount: €6.90</p><p>Barcode</p><p>Size</p><p>STUDENT CARD 10%</p><p>€62.10</p><p>** Sale **</p><p>SCHUH PATASOLE 11/12</p><p>: 7203210090011.</p><p>: One size</p><p>Qty: 1 @ Price : €3.75</p><p>Less Discount Amount: €0.38</p><p>Barcode</p><p>Size</p><p>STUDENT CARD 10%</p><p>€3.37</p><p>Total Amount:</p><p>€144.67</p><p>Amount Tendered:</p><p>€144.67</p><p>Change:</p><p>€0.00</p><p>Date: 27-1-2020</p><p>Time: 14:23:18</p></td></tr></table>"))

Image,Extracted Text
,"schuh47-48 OConnell StreetDublin, D01 PK22TEL No: 01 2476147VAT No: 8272987TCashier : 011: SophieAssistant : 011: Sophie** Sale **ADI GAZELLE:3415691024506BarcodeSize: UK 11Qty: 1 @ Price : €88.00Less Discount Amount: €8.80STUDENT CARD 10%€79.20** Sale **CONS CTAS RENEW HI: 3415955074506: UK 11Qty: 1 @ Price : €69.00Less Discount Amount: €6.90BarcodeSizeSTUDENT CARD 10%€62.10** Sale **SCHUH PATASOLE 11/12: 7203210090011.: One sizeQty: 1 @ Price : €3.75Less Discount Amount: €0.38BarcodeSizeSTUDENT CARD 10%€3.37Total Amount:€144.67Amount Tendered:€144.67Change:€0.00Date: 27-1-2020Time: 14:23:18"


Note: See VisionReceiptImagesSystemTest.py for code that obtained text from Image using Google Cloud API

The image here is quite a high quality email receipt. While this image sucessfully mines the text from the image to a very high degree of accuracy, it is in quite a unqiue format, as are most receipts. The difficulty here is to try and extract the specific data that was outlined as required earlier and to persist that data into a consistent schema. This is essential for proper analysis to be conducted on the data.

### Topic Modelling

The textual description of the line items within the dataset can be used to create topic models on the data. These models will require a vectorised version of that textual data. This can be done through through TF-IDF and Count Vectorisation (see implementation section of report for details on these processes).

The text will be first preprocessed to remove stopwords, turned to lower case, create bigrams, and lemmatize the text. These processes aid the unsupervised learning algorithms. The goal in the inital transformation of the data will be to simplify the text enough so that the algorithms can link different variations of a turn by examining their lemma, or simplest form word. (see lemmatization section in Implementation of the report).

The training alogorithms then use the vectorised form of the text to create models. The algorithms which were used were:
- Latent Dirichlet Allocation (suited for textual topic modelling)
- Non Negative Matrix
- KMeans 
- Self-Organised Maps

The main findings of these algorthms wil be shown in the technical report. See below for an example of the LDA Model and a Visualisation of the Topics

#### Latent Dirichlet Allocation

In [36]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
data = pd.read_csv('LineItemData.csv', encoding='utf-8')

#print(data.head())

data = data['Description'].dropna()

#print(data.head())
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
#pprint(data[:1])

def sent_to_words(sentences):
    for sentence in sentences:
                yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
#print(data_words[:1])

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
##print(trigram_mod[bigram_mod[data_words[0]]])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[['health', 'beauty']]
[[(0, 1), (1, 1)]]


[[('beauty', 1), ('health', 1)]]

In [37]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

[(0,
  '0.440*"warm" + 0.330*"cover" + 0.075*"cushion" + 0.026*"cafe" + '
  '0.024*"local" + 0.020*"honeycomb" + 0.015*"passport" + 0.015*"ball" + '
  '0.012*"candy" + 0.009*"tropical"'),
 (1,
  '0.256*"stamp" + 0.230*"stock" + 0.101*"asstd" + 0.082*"glider" + '
  '0.073*"wish" + 0.054*"good" + 0.047*"stool" + 0.033*"pop" + 0.033*"monkey" '
  '+ 0.020*"mum"'),
 (2,
  '0.572*"home" + 0.277*"sweet" + 0.107*"fancy" + 0.015*"foot" + 0.007*"shelf" '
  '+ 0.005*"carrier" + 0.004*"sheet" + 0.000*"hook" + 0.000*"welcome" + '
  '0.000*"word"'),
 (3,
  '0.052*"set" + 0.038*"heart" + 0.032*"vintage" + 0.029*"bag" + '
  '0.026*"design" + 0.024*"light" + 0.022*"bottle" + 0.021*"hot" + '
  '0.020*"water" + 0.020*"pink"'),
 (4,
  '0.211*"food" + 0.202*"container" + 0.198*"charm" + 0.150*"raffia" + '
  '0.056*"phone" + 0.042*"crystal" + 0.027*"frog" + 0.022*"summer" + '
  '0.022*"daisie" + 0.014*"vine"'),
 (5,
  '0.201*"tin" + 0.104*"calm" + 0.077*"travel" + 0.064*"kneel" + 0.059*"tube" '
  '+ 0.054*"

In [38]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.257333 -0.190463       1        1  71.239891
6     -0.107701  0.386049       2        1  21.465414
5     -0.360954 -0.227209       3        1   3.854026
0     -0.055020 -0.022770       4        1   1.040609
2      0.009947 -0.002019       5        1   0.834462
7      0.008295 -0.002863       6        1   0.683078
1      0.118828  0.028073       7        1   0.447196
4      0.129273  0.031202       8        1   0.435319, topic_info=          Term          Freq         Total Category  logprob  loglift
194        tin  10028.000000  10028.000000  Default  30.0000  30.0000
4         home   6172.000000   6172.000000  Default  29.0000  29.0000
27        warm   5915.000000   5915.000000  Default  28.0000  28.0000
35       metal  14906.000000  14906.000000  Default  27.0000  27.0000
120      small  13751.000000  13751.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
314    sticker      0.153058   1748.542358   Topic8 -10.5125  -3.9066
34        love      0.152778  10432.864258   Topic8 -10.5144  -5.6946
83         bag      0.152743  26782.722656   Topic8 -10.5146  -6.6377
13       white      0.152593   5800.177734   Topic8 -10.5156  -5.1088
53   retrospot      0.152559   7865.438965   Topic8 -10.5158  -5.4136

[358 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
391       3  0.997535        aid
1086      7  0.966650  aluminium
347       3  0.999534     animal
599       7  0.999123      asstd
753       6  0.996811       back
...     ...       ...        ...
168       2  0.999829       wood
80        1  0.999953     wooden
348       3  0.999083   woodland
33        1  0.998863       word
291       2  0.999839       wrap

[197 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 6, 1, 3, 8, 2, 5])